# RILA EDA: Competitive Rate Analysis - REFACTORED

**Refactored:** 2026-01-28  
**Original:** notebooks/rila/02_EDA_rates_RILA.ipynb  

**Changes:**
- Migrated from helpers.* to src.* imports
- Added canonical sys.path auto-detection
- Improved cell structure and documentation
- Preserved exploratory flexibility
- Added validation checkpoints

**Purpose:** Exploratory analysis of WINK competitive rate data for RILA products including rate filtering, time series pivoting, market share weighting, competitive landscape analysis, and rate change tracking.

**Dependencies:** None (independent of sales notebook)

**Note:** EDA notebook - mathematical equivalence not required, exploratory flexibility preserved

## Table of Contents
* [Section 1: Load WINK Data](#sec1:Load)
* [Section 2: Filter Products for RILA Comparables](#sec2:filter)
* [Section 3: Pivot and Apply Market Share Weighting](#sec3:pivot)
* [Section 4: Recent Rate Comparison](#sec4:recent)
* [Section 5: Historical Rate Trends](#sec5:trends)

In [ ]:
%%capture# =============================================================================# STANDARD SETUP CELL - Clean Dependency Pattern# =============================================================================# Standard library importsimport sysimport osfrom pathlib import Pathimport pandas as pdimport numpy as npimport warningsimport matplotlib.pyplot as pltimport seaborn as snsfrom datetime import datetime# Suppress warnings for clean outputwarnings.filterwarnings("ignore")# Canonical sys.path setup (auto-detect project root)# Canonical sys.path setup (auto-detect project root)
# Auto-detect project root (handles actual directory structure)
cwd = os.getcwd()

# Check for actual directory structure
if 'notebooks/production/rila' in cwd:
    project_root = Path(cwd).parents[2]
elif 'notebooks/production/fia' in cwd:
    project_root = Path(cwd).parents[2]
elif 'notebooks/eda/rila' in cwd:
    project_root = Path(cwd).parents[2]
elif 'notebooks/archive' in cwd:
    project_root = Path(cwd).parents[2]
elif os.path.basename(cwd) == 'notebooks':
    project_root = Path(cwd).parent
else:
    project_root = Path(cwd)

project_root = str(project_root)

# IMPORTANT: Verify import will work
if not os.path.exists(os.path.join(project_root, 'src')):
    raise RuntimeError(
        f"sys.path setup failed: 'src' package not found at {project_root}/src\n"
        f"Current directory: {cwd}\n"
        "This indicates the sys.path detection logic needs adjustment."
    )

sys.path.insert(0, project_root)

# Refactored imports (src.* pattern)from src.data import extraction as extfrom src.data import pipelinesfrom src.data.dvc_manager import save_dataset# Visualization themesns.set_theme(style="whitegrid", palette="deep")print("✓ Dependencies loaded successfully")

In [ ]:
# =============================================================================
# AWS CONFIGURATION - Reuse from 00_data_pipeline pattern
# =============================================================================

aws_config = {
    'xid': "x259830",
    'role_arn': "arn:aws:iam::159058241883:role/isg-usbie-annuity-CA-s3-sharing",
    'sts_endpoint_url': "https://sts.us-east-1.amazonaws.com",
    'source_bucket_name': "pruvpcaws031-east-isg-ie-lake",
    'output_bucket_name': "cdo-annuity-364524684987-bucket",
    'output_base_path': "ANN_Price_Elasticity_Data_Science"
}

# Product parameters
version = "v2_0"

# Date parameters
current_time = datetime.now()
current_date = current_time.strftime("%Y-%m-%d")

# FlexGuard competitor product IDs (for market share weighting)
flexguard_product_ids = {
    "Prudential": [2979],
    "Allianz": [2162, 3699],
    "Athene": [2772, 3409],
    "Brighthouse": [2319, 4149],
    "Equitable": [2286, 3282, 3853],
    "Jackson": [3714, 3351],
    "Lincoln": [2358, 4058, 4346],
    "Symetra": [3263, 3751],
    "Trans": [3495],
}

competitors = [
    "Allianz",
    "Athene",
    "Brighthouse",
    "Equitable",
    "Jackson",
    "Lincoln",
    "Symetra",
    "Trans",
]

print(f"✓ Configuration loaded")
print(f"  Version: {version}")
print(f"  Current date: {current_date}")
print(f"  Competitors tracked: {len(competitors)}")

## Section 1: Load WINK Data <a id="sec1:Load"></a>

**Business Purpose**: Load competitive rate data from WINK system for market analysis

**Data Source**: S3 parquet files from `access/ierpt/wink_ann_product_rates/`

In [ ]:
# =============================================================================
# AWS CONNECTION SETUP
# =============================================================================

# AWS Connection using refactored extraction module
sts_client = ext.setup_aws_sts_client_with_validation(aws_config)
assumed_role = ext.assume_iam_role_with_validation(sts_client, aws_config)
s3_resource, bucket = ext.setup_s3_resource_with_validation(
    assumed_role['Credentials'],
    aws_config['source_bucket_name']
)

print(f"✓ AWS connection established")
print(f"  Role: {aws_config['xid']}")
print(f"  Bucket: {aws_config['source_bucket_name']}")

In [ ]:
# =============================================================================
# LOAD WINK DATA - Using refactored extraction
# =============================================================================

# Load all WINK competitive rate data
df_wink = ext.discover_and_load_wink_data(
    bucket, 
    s3_resource, 
    aws_config['source_bucket_name']
)

print(f"✓ WINK data loaded")
print(f"  Total records: {len(df_wink):,}")
print(f"  Total columns: {df_wink.shape[1]}")

In [ ]:
# =============================================================================
# INITIAL WINK PROCESSING
# =============================================================================

# Create date column
df_wink["date"] = pd.to_datetime(df_wink["effectiveDate"])

# Store as history dataframe
df_WINK_history = df_wink.copy()

# Standardize text fields
df_WINK_history["companyName"] = df_WINK_history.companyName.str.strip().str.lower()
df_WINK_history["productName"] = df_WINK_history.productName.str.strip().str.lower()

# Round rate columns
df_WINK_history["bufferRate"] = np.round(df_WINK_history["bufferRate"], 3)
df_WINK_history["participationRate"] = np.round(df_WINK_history["participationRate"], 3)
df_WINK_history["spreadRate"] = np.round(df_WINK_history["spreadRate"], 3)
df_WINK_history["performanceTriggeredRate"] = np.round(
    df_WINK_history["performanceTriggeredRate"], 3
)

# Remove records with missing dates
df_WINK_history = df_WINK_history[~df_WINK_history.date.isna()]

max_date = df_WINK_history.date.max()

print(f"✓ Initial processing complete")
print(f"  Date range: {df_WINK_history.date.min()} to {max_date}")
print(f"  Companies: {df_WINK_history.companyName.nunique()}")

In [ ]:
# =============================================================================
# VALIDATION CHECKPOINT
# =============================================================================

# Validate data quality
assert not df_WINK_history.empty, "DataFrame is empty"
assert len(df_WINK_history) > 1000, f"Expected >1000 rows, got {len(df_WINK_history)}"
assert 'date' in df_WINK_history.columns, "Missing date column"
assert 'capRate' in df_WINK_history.columns, "Missing capRate column"
assert 'companyName' in df_WINK_history.columns, "Missing companyName column"

print(f"✓ Data validation passed")
print(f"  Records: {len(df_WINK_history):,}")
print(f"  Columns: {df_WINK_history.shape[1]}")
print(f"  Date range: {df_WINK_history.date.min()} to {df_WINK_history.date.max()}")
print(f"  Missing values: {df_WINK_history.isna().sum().sum()}")

## Section 2: Filter Products for RILA Comparables <a id="sec2:filter"></a>

Filter WINK data for products comparable to FlexGuard 6Y20B:
- RILA (Structured) products only
- S&P 500 indexed
- 10% buffer rate
- 1-year term
- Cap rate products (not participation rate)
- No fee products

In [ ]:
# =============================================================================
# PRODUCT FILTERING - RILA Comparables
# =============================================================================

# Select columns for analysis
columns_for_analysis = [
    "date",
    "capRate",
    "companyName",
    "effectiveDate",
    "productID",
    "productName",
    "amBestRating",
    "surrChargeDuration",
]

# Apply filters (inline for EDA flexibility)
df_WINK_history = df_WINK_history[df_WINK_history["productTypeName"] == "Structured"]  # Only RILAs
df_WINK_history = df_WINK_history[df_WINK_history["indexUsed"] == "S&P 500"]  # Only SP500
df_WINK_history = df_WINK_history[df_WINK_history["annualFeeForIndexingMethod"].isna()]  # Remove fee products
df_WINK_history = df_WINK_history[df_WINK_history["capRate"].notna()]  # Only cap rate products
df_WINK_history = df_WINK_history[df_WINK_history["participationRate"] == 1.00]  # Participation rate = 1
df_WINK_history = df_WINK_history[df_WINK_history["bufferRate"] == 0.10]  # 10% buffer
df_WINK_history = df_WINK_history[df_WINK_history["bufferModifier"] == "Losses Covered Up To"]  # Buffer modifier
df_WINK_history = df_WINK_history[df_WINK_history["indexCreditingFrequency"] == "Annual"]  # 1-year term
df_WINK_history = df_WINK_history[df_WINK_history["indexingMethod"] == "Annual PTP"]  # Point-to-point
df_WINK_history = df_WINK_history[
    df_WINK_history["defaultActuarialView"] == "Annual PTP: Annual Crediting, No Premium Bonus"
]  # Standard view
df_WINK_history = df_WINK_history[df_WINK_history["spreadRate"].isna()]  # No bonus structures

# Remove duplicates
df_WINK_history = df_WINK_history.sort_values(
    by=["date", "capRate"], ascending=[True, False]
).drop_duplicates(subset=["date", "companyName", "productName", "productID", "mva"])

# Select final columns
df_WINK_history = df_WINK_history[columns_for_analysis]

print(f"✓ Product filtering complete")
print(f"  Filtered records: {len(df_WINK_history):,}")
print(f"  Date range: {df_WINK_history.date.min()} to {df_WINK_history.date.max()}")
print(f"  Products: {df_WINK_history.productID.nunique()}")

In [ ]:
# Inspect specific competitor (example: Brighthouse)
df_brighthouse = df_WINK_history[df_WINK_history.productID.isin([2319, 4149])]

df_brighthouse["Date"] = df_brighthouse["date"].apply(lambda x: str(x)[:11])
print("\nBrighthouse rate history:")
print(
    df_brighthouse[["Date", "capRate", "productName", "companyName", "productID"]]
    .set_index("Date")
    .tail(10)
    .to_markdown()
)

## Section 3: Pivot and Apply Market Share Weighting <a id="sec3:pivot"></a>

Transform data from long to wide format with competitors as columns, then apply market share weighting

In [ ]:
# =============================================================================
# WINK RATE PROCESSING - Using refactored pipeline
# =============================================================================

# Configure WINK processing (inline for EDA flexibility)
start_date = "2018-06-21"

wink_config = {
    'product_ids': flexguard_product_ids,
    'start_date': start_date,
    'rolling_window': 1,  # No rolling average for raw data
    'filter_criteria': {
        'productTypeName': 'Structured',
        'indexUsed': 'S&P 500',
        'bufferRate': 0.10
    }
}

# Apply WINK processing using refactored pipeline
df_wink_rates = pipelines.apply_wink_rate_processing(df_WINK_history, wink_config)

print(f"✓ WINK rate processing complete")
print(f"  Time series records: {len(df_wink_rates):,}")
print(f"  Competitor columns: {[col for col in df_wink_rates.columns if col in competitors + ['Prudential']]}")

In [ ]:
# =============================================================================
# MARKET SHARE WEIGHTING - Using refactored pipeline
# =============================================================================

# Load market share weights from S3
market_share_s3_path = "s3://cdo-annuity-364524684987-bucket/ANN_Price_Elasticity_Data_Science/flex_guard_market_share_2025_10_01.parquet"
df_quarter_weights = ext.load_market_share_weights_from_s3(market_share_s3_path)

print(f"Market share weights loaded: {df_quarter_weights.shape}")

# Apply market share weighting (creates C_weighted_mean)
df_wink_rates = pipelines.apply_market_share_weighting(df_wink_rates, df_quarter_weights)

print(f"✓ Market share weighting applied")
print(f"  Weighted features created: {[col for col in df_wink_rates.columns if col.startswith('C_')]}")

In [ ]:
# =============================================================================
# SAVE COMPETITIVE RATES - Using DVC for tracking (optional for EDA)
# =============================================================================

# Save competitive rates
save_dataset(df_wink_rates, "WINK_competitive_landscape_1Y10_EDA")

print(f"✓ Competitive rates saved")

## Section 4: Recent Rate Comparison <a id="sec4:recent"></a>

Analyze recent rate changes across competitors

In [ ]:
# =============================================================================
# RECENT RATE COMPARISON
# =============================================================================

col = ["Prudential", "C_weighted_mean"] + competitors
df_wink_rates["day"] = df_wink_rates["date"].apply(
    lambda x: str(x)[5:10].replace("-", "/")
)

print("\nRecent rate levels (last 30 days):")
print(
    df_wink_rates[["day"] + col]
    .set_index("day")
    .tail(30)
    .to_markdown(tablefmt="fancy_grid", floatfmt=".2f", numalign="center")
)

In [ ]:
# =============================================================================
# RATE CHANGES (basis points)
# =============================================================================

df_wink_diff = df_wink_rates.tail(30).copy()
df_wink_diff.loc[:, col] = (df_wink_rates[col].diff() * 100).apply(
    lambda x: np.round(x)
)

df_wink_diff.loc[:, "s"] = df_wink_diff[col].sum(axis=1)
df_wink_diff.loc[:, col] = df_wink_diff.loc[:, col].astype("string") + "bps"
df_wink_diff.loc[:, col] = df_wink_diff.loc[:, col].apply(
    lambda x: x.str.replace(".0", "").str.replace(".", "")
)
df_wink_diff.loc[:, col] = np.where(
    df_wink_diff[col].values == "0bps", "", df_wink_diff[col].values
)

print("\nRate changes (last 30 days, basis points):")
print(
    df_wink_diff[["day"] + col]
    .set_index("day")
    .to_markdown(tablefmt="fancy_grid", numalign="center")
)

## Section 5: Historical Rate Trends <a id="sec5:trends"></a>

Visualize competitive rate history over time

In [ ]:
# =============================================================================
# FULL HISTORY VISUALIZATION
# =============================================================================

c = [
    "Prudential",
    "Athene",
    "Brighthouse",
    "Allianz",
    "Equitable",
    "Jackson",
    "Lincoln",
    "Symetra",
    "Trans",
]

df = df_wink_rates[["date"] + c].dropna(subset="Prudential")
figure, axes = plt.subplots(1, 1, sharex=False, sharey=False, figsize=(16, 6))
axes.set_title("RILA 1Y10 Rates - Full History")

sns.lineplot(
    df.melt(id_vars="date"), 
    x="date", 
    y="value", 
    hue="variable", 
    ax=axes, 
    linewidth=2
)

sns.lineplot(
    df_wink_rates.dropna(subset="Prudential"),
    x="date",
    y="raw_median",
    color="k",
    linewidth=5,
    label="median",
    linestyle="dashed",
    ax=axes
)

plt.xlabel("Date")
plt.ylabel("Cap Rate")
plt.tight_layout()
plt.savefig("RILA_1Y10_full_history.png")
plt.show()

print("✓ Full history visualization saved to RILA_1Y10_full_history.png")

In [ ]:
# =============================================================================
# RECENT HISTORY VISUALIZATION (since Oct 2024)
# =============================================================================

c = [
    "Prudential",
    "Allianz",
    "Brighthouse",
    "Equitable",
    "Jackson",
    "Lincoln",
    "Symetra",
    "Trans",
]

cutoff = "2024-10-01"
df = df_wink_rates[df_wink_rates["date"] > cutoff][["date"] + c].dropna(
    subset="Prudential"
)

figure, axes = plt.subplots(1, 1, sharex=False, sharey=False, figsize=(16, 6))
axes.set_title(f"RILA 1Y10 Rates - Since {cutoff}")

sns.lineplot(
    df.melt(id_vars="date"),
    x="date",
    y="value",
    hue="variable",
    ax=axes,
    linewidth=4,
    linestyle="dashed",
)

sns.lineplot(
    df_wink_rates[df_wink_rates["date"] > cutoff].dropna(subset="Prudential"),
    x="date",
    y="raw_median",
    color="k",
    linewidth=5,
    label="median",
    linestyle="dashed",
    ax=axes
)

plt.xlabel("Date")
plt.ylabel("Cap Rate")
plt.tight_layout()
plt.savefig("RILA_1Y10_recent_history.png")
plt.show()

print(f"✓ Recent history visualization saved to RILA_1Y10_recent_history.png")

---

## EDA Complete

**Key Findings:**
- Competitive rate landscape analyzed for 8 major competitors
- Market share weighted averages computed
- Recent rate changes tracked and quantified
- Historical trends visualized

**Next Steps:** 
- Proceed to 03_EDA_RILA_feature_engineering.ipynb to combine sales and rates
- Create lag features and spreads for modeling